In [7]:
import sys
sys.path.append('/host/d/Github/')
import nibabel as nb
import glob
import os
import glob
import lpips
import torch
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity

import Diffusion_denoising_thin_slice.functions_collection as ff
import Diffusion_denoising_thin_slice.Build_lists.Build_list as Build_list
import Diffusion_denoising_thin_slice.Data_processing as Data_processing

In [17]:
build_sheet =  Build_list.Build(os.path.join('/host/d/Data/low_dose_CT/Patient_lists/mayo_low_dose_CT_gaussian_simulation_v2.xlsx'))
batch_list, patient_id_list, random_num_list,noise_file_all_list, noise_file_odd_list, noise_file_even_list, ground_truth_file_list, slice_num_list = build_sheet.__build__(batch_list = ['test']) 
n = ff.get_X_numbers_in_interval(total_number = patient_id_list.shape[0],start_number = 0,end_number = 1, interval = 1)

In [9]:
def calc_mae_with_ref_window(img, ref, vmin, vmax):
    maes = []
    for slice_num in range(0, img.shape[-1]):
        slice_img = img[:,:,slice_num]
        slice_ref = ref[:,:,slice_num]
        mask = np.where((slice_ref >= vmin) & (slice_ref <= vmax), 1, 0)
        mae = np.sum(np.abs(slice_img - slice_ref) * mask) / np.sum(mask)
        maes.append(mae)

    return np.mean(maes), np.std(maes)

In [10]:
def calc_ssim_with_ref_window(img, ref, vmin, vmax):

    ssims = []
    for slice_num in range(0, img.shape[-1]):
        slice_img = img[:,:,slice_num]
        slice_ref = ref[:,:,slice_num]
        mask = np.where((slice_ref >= vmin) & (slice_ref <= vmax), 1, 0)
        _, ssim_map = structural_similarity(slice_img, slice_ref, data_range=vmax - vmin, full=True)
        ssim = np.sum(ssim_map * mask) / np.sum(mask)
        ssims.append(ssim)

    return np.mean(ssims), np.std(ssims)

In [11]:
def calc_lpips(imgs1, imgs2, vmin, vmax):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    loss_fn = lpips.LPIPS().to(device)
    
    lpipss = []
    for slice_num in range(0, imgs1.shape[-1]):
        slice1 = imgs1[:,:,slice_num]
        slice2 = imgs2[:,:,slice_num]

        slice1 = np.clip(slice1, vmin, vmax).astype(np.float32)
        slice2 = np.clip(slice2, vmin, vmax).astype(np.float32)

        slice1 = (slice1 - vmin) / (vmax - vmin) * 2 - 1
        slice2 = (slice2 - vmin) / (vmax - vmin) * 2 - 1

        slice1 = np.stack([slice1, slice1, slice1], axis=-1)
        slice2 = np.stack([slice2, slice2, slice2], axis=-1)
        # print('after stack, slice1 shape:', slice1.shape, ' slice2 shape:', slice2.shape)

        slice1 = np.transpose(slice1, (2, 0, 1))[np.newaxis, ...]
        slice2 = np.transpose(slice2, (2, 0, 1))[np.newaxis, ...]
        # print('after transpose, slice1 shape:', slice1.shape, ' slice2 shape:', slice2.shape)

        slice1 = torch.from_numpy(slice1).to(device)
        slice2 = torch.from_numpy(slice2).to(device)

        lpips_val = loss_fn(slice1, slice2)
        lpipss.append(lpips_val.item())

      

    return np.mean(lpipss), np.std(lpipss)


In [25]:
## metric calculations
results = []
for i in range(0,n.shape[0]):
    patient_id = patient_id_list[n[i]]
    random_n = random_num_list[n[i]]
    print(patient_id,  random_n)

    # reference image
    gt_file = os.path.join('/host/d/Data/low_dose_CT/nii_imgs', patient_id,  'img.nii.gz')
    gt_img = nb.load(gt_file).get_fdata()[...,100:200]

    # noisy image
    condition_file = os.path.join('/host/d/Data/low_dose_CT/simulation_v2', patient_id,'gaussian_random_'+str(random_n), 'recon_all.nii.gz')
    condition_img = nb.load(condition_file).get_fdata()[...,100:200]

    # noise2noise
    noise2noise_file = os.path.join('/host/d/projects/denoising/models/noise2noise/pred_images_input_both', patient_id,'random_'+str(random_n), 'epoch80/pred_img.nii.gz')
    noise2noise_img = nb.load(noise2noise_file).get_fdata()[...,100:200]

    # supervised method
    supervised_file = os.path.join('/host/d/projects/denoising/models/supervised_poisson_2/pred_images',patient_id,  'random_'+str(random_n), 'epoch114_1/pred_img.nii.gz')
    supervised_img = nb.load(supervised_file).get_fdata()[...,100:200]

    # our method (unsupervised), beta = 0, 1 inference
    unsupervised_beta0_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_2/pred_images_input_both', patient_id, 'random_'+str(random_n), 'epoch355_1/pred_img.nii.gz')
    unsupervised_beta0_img = nb.load(unsupervised_beta0_file).get_fdata()

    # our method (unsupervised), beta = 0, 10 inference
    unsupervised_beta0_avg10_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_2/pred_images_input_both', patient_id,'random_'+str(random_n), 'epoch355avg/pred_img_scans10.nii.gz')
    unsupervised_beta0_avg10_img = nb.load(unsupervised_beta0_avg10_file).get_fdata()

    # # our method (unsupervised), beta = 0, 20 inference
    unsupervised_beta0_avg20_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_2/pred_images_input_both', patient_id,'random_'+str(random_n), 'epoch355avg/pred_img_scans20.nii.gz')
    unsupervised_beta0_avg20_img = nb.load(unsupervised_beta0_avg20_file).get_fdata()

    # # our method (distilled DDPM)
    # distilled_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_gaussian_beta0_distilled/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch227_1/pred_img.nii.gz')
    # distilled_img = nb.load(distilled_file).get_fdata()

    # # our method (distilled DDPM + regularization)
    # distilled_reg_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_gaussian_beta0_distilled_Lpips0.1_Edge0.05/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch271_1/pred_img.nii.gz')
    # distilled_reg_img = nb.load(distilled_reg_file).get_fdata()

    # # our method (distilled DDPM + regularization weight 0.2)
    # distilled_reg2_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_gaussian_beta0_distilled_Lpips0.2_Edge0.05/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch267_1/pred_img.nii.gz')
    # distilled_reg2_img = nb.load(distilled_reg2_file).get_fdata()

    # # our method (distilled_EDM)
    # distilled_EDM_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_gaussian_beta0_distilled_EDM/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch160_1/pred_img.nii.gz')
    # distilled_EDM_img = nb.load(distilled_EDM_file).get_fdata()

    # # our method(distilled EDM + regularization)
    # distilled_EDM_reg_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_gaussian_beta0_distilled_EDM_Lpips0.2_Edge0.1/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch119_1/pred_img.nii.gz')
    # distilled_EDM_reg_img = nb.load(distilled_EDM_reg_file).get_fdata()

    ## calculate metrics
    # MAE
    vmin = -200
    vmax = 250
    mae_condition, mae_condition_std = calc_mae_with_ref_window(condition_img, gt_img, vmin, vmax)
    mae_noise2noise, mae_noise2noise_std = calc_mae_with_ref_window(noise2noise_img, gt_img, vmin, vmax)
    mae_supervised, mae_supervised_std = calc_mae_with_ref_window(supervised_img, gt_img, vmin, vmax)
    mae_unsupervised_beta0, mae_unsupervised_beta0_std = calc_mae_with_ref_window(unsupervised_beta0_img, gt_img, vmin, vmax)
    mae_unsupervised_beta0_avg10, mae_unsupervised_beta0_avg10_std = calc_mae_with_ref_window(unsupervised_beta0_avg10_img, gt_img, vmin, vmax)
    mae_unsupervised_beta0_avg20, mae_unsupervised_beta0_avg20_std = calc_mae_with_ref_window(unsupervised_beta0_avg20_img, gt_img, vmin, vmax)
    # mae_distilled, mae_distilled_std = calc_mae_with_ref_window(distilled_img, gt_img, vmin, vmax)
    # mae_distilled_reg, _ = calc_mae_with_ref_window(distilled_reg_img, gt_img, vmin, vmax)
    # mae_distilled_reg2, _ = calc_mae_with_ref_window(distilled_reg2_img, gt_img, vmin, vmax)
    # mae_distilled_EDM, mae_distilled_EDM_std = calc_mae_with_ref_window(distilled_EDM_img, gt_img, vmin, vmax)
    # mae_distilled_EDM_reg, mae_distilled_EDM_reg_std = calc_mae_with_ref_window(distilled_EDM_reg_img, gt_img, vmin, vmax)
    
    # # SSIM
    ssim_condition, ssim_condition_std = calc_ssim_with_ref_window(condition_img, gt_img, vmin, vmax)
    ssim_noise2noise, ssim_noise2noise_std = calc_ssim_with_ref_window(noise2noise_img, gt_img, vmin, vmax)
    ssim_supervised, ssim_supervised_std = calc_ssim_with_ref_window(supervised_img, gt_img, vmin, vmax)
    ssim_unsupervised_beta0, ssim_unsupervised_beta0_std = calc_ssim_with_ref_window(unsupervised_beta0_img, gt_img, vmin, vmax)
    ssim_unsupervised_beta0_avg10, ssim_unsupervised_beta0_avg10_std = calc_ssim_with_ref_window(unsupervised_beta0_avg10_img, gt_img, vmin, vmax)
    ssim_unsupervised_beta0_avg20, ssim_unsupervised_beta0_avg20_std = calc_ssim_with_ref_window(unsupervised_beta0_avg20_img, gt_img, vmin, vmax)
    # ssim_distilled, ssim_distilled_std = calc_ssim_with_ref_window(distilled_img, gt_img, vmin, vmax)
    # ssim_distilled_reg,_ = calc_ssim_with_ref_window(distilled_reg_img, gt_img, vmin, vmax)
    # ssim_distilled_reg2,_ = calc_ssim_with_ref_window(distilled_reg2_img, gt_img, vmin, vmax)
    # ssim_distilled_EDM, ssim_distilled_EDM_std = calc_ssim_with_ref_window(distilled_EDM_img, gt_img, vmin, vmax)
    # ssim_distilled_EDM_reg, ssim_distilled_EDM_reg_std = calc_ssim_with_ref_window(distilled_EDM_reg_img, gt_img, vmin, vmax)

    # # lpips
    lpips_condition, _ = calc_lpips(condition_img, gt_img, vmin, vmax)
    lpips_noise2noise, _ = calc_lpips(noise2noise_img, gt_img, vmin, vmax)
    lpips_supervised, _ = calc_lpips(supervised_img, gt_img, vmin, vmax)
    lpips_unsupervised_beta0, _ = calc_lpips(unsupervised_beta0_img, gt_img, vmin, vmax)
    lpips_unsupervised_beta0_avg10, _ = calc_lpips(unsupervised_beta0_avg10_img, gt_img, vmin, vmax)
    lpips_unsupervised_beta0_avg20, _ = calc_lpips(unsupervised_beta0_avg20_img, gt_img, vmin, vmax)
    # lpips_distilled, _ = calc_lpips(distilled_img, gt_img, vmin, vmax)
    # lpips_distilled_reg, _ = calc_lpips(distilled_reg_img, gt_img, vmin, vmax)
    # lpips_distilled_reg2, _ = calc_lpips(distilled_reg2_img, gt_img, vmin, vmax)
    # lpips_distilled_EDM, _ = calc_lpips(distilled_EDM_img, gt_img, vmin, vmax)
    # lpips_distilled_EDM_reg, _ = calc_lpips(distilled_EDM_reg_img, gt_img, vmin, vmax)

    print('condition: ', mae_condition, ssim_condition, lpips_condition)
    print('noise2noise: ', mae_noise2noise, ssim_noise2noise, lpips_noise2noise)
    print('supervised: ', mae_supervised, ssim_supervised, lpips_supervised)
    print('unsupervised_beta0: ', mae_unsupervised_beta0, ssim_unsupervised_beta0, lpips_unsupervised_beta0)
    print('unsupervised_beta0_avg10: ', mae_unsupervised_beta0_avg10, ssim_unsupervised_beta0_avg10, lpips_unsupervised_beta0_avg10)
    print('unsupervised_beta0_avg20: ', mae_unsupervised_beta0_avg20, ssim_unsupervised_beta0_avg20, lpips_unsupervised_beta0_avg20)
    # print('distilled: ', mae_distilled, ssim_distilled, lpips_distilled)
    # print('distilled_reg: ', mae_distilled_reg, ssim_distilled_reg, lpips_distilled_reg)
    # print('distilled_reg2: ', mae_distilled_reg2, ssim_distilled_reg2, lpips_distilled_reg2)
    # print('distilled_EDM: ', mae_distilled_EDM, ssim_distilled_EDM, lpips_distilled_EDM)
    # print('distilled_EDM_reg: ', mae_distilled_EDM_reg, ssim_distilled_EDM_reg, lpips_distilled_EDM_reg)

    results.append([patient_id, random_n,
                    mae_condition,mae_noise2noise,mae_supervised,mae_unsupervised_beta0,mae_unsupervised_beta0_avg10,mae_unsupervised_beta0_avg20,
                  
                    ssim_condition,ssim_noise2noise,ssim_supervised,ssim_unsupervised_beta0,ssim_unsupervised_beta0_avg10,ssim_unsupervised_beta0_avg20,
                   
                    lpips_condition,lpips_noise2noise,lpips_supervised,lpips_unsupervised_beta0,lpips_unsupervised_beta0_avg10,lpips_unsupervised_beta0_avg20,
                    ])
    dd = pd.DataFrame(results, columns = ['patient_id', 'random_n',
                                          'mae_condition','mae_noise2noise','mae_supervised','mae_unsupervised_beta0','mae_unsupervised_beta0_avg10','mae_unsupervised_beta0_avg20',
                                            'ssim_condition','ssim_noise2noise','ssim_supervised','ssim_unsupervised_beta0','ssim_unsupervised_beta0_avg10','ssim_unsupervised_beta0_avg20',
                                            'lpips_condition','lpips_noise2noise','lpips_supervised','lpips_unsupervised_beta0','lpips_unsupervised_beta0_avg10','lpips_unsupervised_beta0_avg20',
                                          ])
                                     
    dd.to_excel('/host/d/projects/denoising/models/unsupervised_gaussian_2/quantitative_results_epoch355.xlsx', index = False)

L192 0
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


/usr/local/lib/python3.8/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_location='cpu'), str

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
condition:  15.035378010512 0.7608820363171143 0.06853045552968978
noise2noise:  11.626823233777266 0.7836563093950122 0.03659829761832953
supervised:  13.042174872435133 0.7480393609295889 0.0371778023801

...

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
condition:  15.459558894566939 0.793931477902283 0.06988112814724445
noise2noise:  12.843069798231772 0.7512453494904208 0.06127353012561798
supervised:  16.079770148553973 0.7153424899237636 0.06738330133

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth


/usr/local/lib/python3.8/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_location='cpu'), str

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
condition:  19.70406450822791 0.7114996472294449 0.09638312734663486
noise2noise:  14.204391003143037 0.7440370382224182 0.05326000511646271
supervised:  17.02438947510002 0.7059313954402399 0.054500395953

In [41]:
# calculate mean
## metric calculations
results = []
for i in range(0,n.shape[0]):
    patient_id = patient_id_list[n[i]]
    patient_subid = patient_subid_list[n[i]]
    random_n = random_num_list[n[i]]
    print(patient_id, patient_subid, random_n)

    # reference image
    gt_file = os.path.join('/mnt/camca_NAS/denoising/models/unsupervised_gaussian_beta0/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/gt_img.nii.gz')
    gt_img = nb.load(gt_file).get_fdata()

    # noisy image
    condition_file = os.path.join('/mnt/camca_NAS/denoising/models/unsupervised_gaussian_beta0/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/condition_img.nii.gz')
    condition_img = nb.load(condition_file).get_fdata()

    # noise2noise
    noise2noise_file = os.path.join('/mnt/camca_NAS/denoising/models/noise2noise/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch78/pred_img.nii.gz')
    noise2noise_img = nb.load(noise2noise_file).get_fdata() 

    # supervised method
    supervised_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_poisson/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch58_1/pred_img.nii.gz')
    supervised_img = nb.load(supervised_file).get_fdata() 

    # our method (unsupervised), beta = 0, 1 inference
    unsupervised_beta0_file = os.path.join('/mnt/camca_NAS/denoising/models/unsupervised_gaussian_beta0/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/pred_img.nii.gz')
    unsupervised_beta0_img = nb.load(unsupervised_beta0_file).get_fdata()

    # our method (unsupervised), beta = 0, 10 inference
    unsupervised_beta0_avg10_file = os.path.join('/mnt/camca_NAS/denoising/models/unsupervised_gaussian_beta0/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61avg/pred_img_scans10.nii.gz')
    unsupervised_beta0_avg10_img = nb.load(unsupervised_beta0_avg10_file).get_fdata()

    # our method (unsupervised), beta = 0, 20 inference
    unsupervised_beta0_avg20_file = os.path.join('/mnt/camca_NAS/denoising/models/unsupervised_gaussian_beta0/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61avg/pred_img_scans20.nii.gz')
    unsupervised_beta0_avg20_img = nb.load(unsupervised_beta0_avg20_file).get_fdata()

    # our method (distilled DDPM)
    distilled_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_gaussian_beta0_distilled/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch227_1/pred_img.nii.gz')
    distilled_img = nb.load(distilled_file).get_fdata()

    # our method (distilled_EDM)
    distilled_EDM_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_gaussian_beta0_distilled_EDM/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch160_1/pred_img.nii.gz')
    distilled_EDM_img = nb.load(distilled_EDM_file).get_fdata()

    # our method(distilled EDM + regularization)
    distilled_EDM_reg_file = os.path.join('/mnt/camca_NAS/denoising/models/supervised_gaussian_beta0_distilled_EDM_Lpips0.2_Edge0.1/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch119_1/pred_img.nii.gz')
    distilled_EDM_reg_img = nb.load(distilled_EDM_reg_file).get_fdata()

    ## calculate main
    x,y = 256,256
    mean_gt = np.mean(np.clip(gt_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_condition = np.mean(np.clip(condition_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_noise2noise = np.mean(np.clip(noise2noise_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_supervised = np.mean(np.clip(supervised_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_unsupervised_beta0 = np.mean(np.clip(unsupervised_beta0_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_unsupervised_beta0_avg10 = np.mean(np.clip(unsupervised_beta0_avg10_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_unsupervised_beta0_avg20 = np.mean(np.clip(unsupervised_beta0_avg20_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_distilled = np.mean(np.clip(distilled_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_distilled_EDM = np.mean(np.clip(distilled_EDM_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_distilled_EDM_reg = np.mean(np.clip(distilled_EDM_reg_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    
    results.append([patient_id, patient_subid, random_n,
                    mean_gt, mean_condition, mean_noise2noise, mean_supervised, mean_unsupervised_beta0, mean_unsupervised_beta0_avg10, mean_unsupervised_beta0_avg20,
                    mean_distilled, mean_distilled_EDM, mean_distilled_EDM_reg])
    dd = pd.DataFrame(results, columns = ['patient_id','patient_subid','random_n',
                                            'mean_gt', 'mean_condition', 'mean_noise2noise', 'mean_supervised', 'mean_unsupervised_beta0', 'mean_unsupervised_beta0_avg10', 'mean_unsupervised_beta0_avg20',
                                            'mean_distilled', 'mean_distilled_EDM', 'mean_distilled_EDM_reg'])

    dd.to_excel('/mnt/camca_NAS/denoising/distilled_mean_results.xlsx', index = False)

00214841 0000455418 0


00105734 0000455323 0
00214867 0000455521 0
00214877 0000455524 0
00214836 0000455414 0
00154137 0000455529 0
00214901 0000455577 0
00174234 0000455725 0
00035838 0000455369 0
00019591 0000455445 0
00214792 0000455334 0
00010461 0000455845 0
00214931 0000455660 0
00010436 0000034479 0
00214878 0000455518 0
00014689 0000455416 0
